# 1. IMPORT the necessary libraries

In [1]:
import pandas as pd
import boto3
import json

# NOTE: AWS secret and Access Key are not available in the dwh.cfg for security reasons.
- To run this code
    - Create a new IAM user in your AWS account
    - provide it AdministratorAccess
    - Take note of the Access Key and secret and enter it in the dwh.cfg 
    
# 2. LOAD DWH Params from a dwh.cfg file

In [4]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('sparkify.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

SPARKIFY_CLUSTER_TYPE       = config.get("SPARKIFY","SPARKIFY_CLUSTER_TYPE")
SPARKIFY_NUM_NODES          = config.get("SPARKIFY","SPARKIFY_NUM_NODES")
SPARKIFY_NODE_TYPE          = config.get("SPARKIFY","SPARKIFY_NODE_TYPE")

SPARKIFY_CLUSTER_IDENTIFIER = config.get("SPARKIFY","SPARKIFY_CLUSTER_IDENTIFIER")
SPARKIFY_DB                 = config.get("SPARKIFY","SPARKIFY_DB")
SPARKIFY_DB_USER            = config.get("SPARKIFY","SPARKIFY_DB_USER")
SPARKIFY_DB_PASSWORD        = config.get("SPARKIFY","SPARKIFY_DB_PASSWORD")
SPARKIFY_PORT               = config.get("SPARKIFY","SPARKIFY_PORT")

SPARKIFY_IAM_ROLE_NAME      = config.get("SPARKIFY", "SPARKIFY_IAM_ROLE_NAME")

(SPARKIFY_DB_USER, SPARKIFY_DB_PASSWORD, SPARKIFY_DB)

pd.DataFrame({"Param":
                  ["SPARKIFY_CLUSTER_TYPE", "SPARKIFY_NUM_NODES", "SPARKIFY_NODE_TYPE", "SPARKIFY_CLUSTER_IDENTIFIER", "SPARKIFY_DB", "SPARKIFY_DB_USER", "SPARKIFY_DB_PASSWORD", "SPARKIFY_PORT", "SPARKIFY_IAM_ROLE_NAME"],
              "Value":
                  [SPARKIFY_CLUSTER_TYPE, SPARKIFY_NUM_NODES, SPARKIFY_NODE_TYPE, SPARKIFY_CLUSTER_IDENTIFIER, SPARKIFY_DB, SPARKIFY_DB_USER, SPARKIFY_DB_PASSWORD, SPARKIFY_PORT, SPARKIFY_IAM_ROLE_NAME]
             })

,Param,Value
0,SPARKIFY_CLUSTER_TYPE,multi-node
1,SPARKIFY_NUM_NODES,4
2,SPARKIFY_NODE_TYPE,dc2.large
3,SPARKIFY_CLUSTER_IDENTIFIER,sparkifyCluster
4,SPARKIFY_DB,dev
5,SPARKIFY_DB_USER,awsuser
6,SPARKIFY_DB_PASSWORD,Passw0rd
7,SPARKIFY_PORT,5439
8,SPARKIFY_IAM_ROLE_NAME,sparkifyRole


# 3. Create clients from IAM, EC2, S3 and Redshift

In [5]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

# 4. IAM Role
- Create an IAM Role that makes Redshift able to access the S3 DBBucket

In [6]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    sparkifyRole = iam.create_role(
        Path='/',
        RoleName=SPARKIFY_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=SPARKIFY_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=SPARKIFY_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::983440384925:role/sparkifyRole


# 6. Redshift Cluster
- Create a RedShift Cluster

In [7]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=SPARKIFY_CLUSTER_TYPE,
        NodeType=SPARKIFY_NODE_TYPE,
        NumberOfNodes=int(SPARKIFY_NUM_NODES),

        #Identifiers & Credentials
        DBName=SPARKIFY_DB,
        ClusterIdentifier=SPARKIFY_CLUSTER_IDENTIFIER,
        MasterUsername=SPARKIFY_DB_USER,
        MasterUserPassword=SPARKIFY_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

# 7. Describe the cluster and show the details
- Run the block of code several times until the cluster status becomes available

In [8]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=SPARKIFY_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,MasterUsername,awsuser
1,NumberOfNodes,4
2,VpcId,vpc-6358f11b
3,DBName,dev
4,ClusterIdentifier,sparkifycluster
5,NodeType,dc2.large
6,ClusterStatus,creating


# 8. Open the incoming TCP port to access the cluster endpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(SPARKIFY_PORT),
        ToPort=int(SPARKIFY_PORT)
    )
except Exception as e:
    print(e)

# Delete the Cluster
It's good to clean up resources that you don't need anymore. Delete the cluster with care, there is no way back.
The command is commented, it's to make sure that you don't delete the cluster by accident

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
#redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

# Check the status of the cluster

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)